In [ ]:
from skimage import data, transform, io
from skimage.color import rgb2gray
import pandas as pd
import matplotlib.pyplot as plt
from MILWRM.MxIF import *

In [ ]:
sample_key = pd.read_csv("../resources/visium_sample_key.csv", index_col=0)

sample_key.ws_adata = sample_key.ws_adata.fillna("")

In [ ]:
# make output directories
import os
if not os.path.exists("MxIF_out"):
    os.mkdir("MxIF_out")

---

## Define functions

In [ ]:
import sys; sys.path.append("../resources/")
from napari_registration_functions import *

In [ ]:
def on_transform_changed_drag(event):
    """record affine transform as ST is manipulated"""
    viewer.layers.selection.active.affine = event.value

---
# Read in two images for registration

In [ ]:
slide_id = "SG00002"
visium_id = "8899_4"

In [ ]:
sample_key.loc[sample_key.patient_name=="SG00002", ["sample_key","ws_he","ws_flip"]]

In [ ]:
%time a = img.from_npz("../data/MxIF/{}_downsample8.npz".format(slide_id))

In [ ]:
rgb = a.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
ST = io.imread("../data/ST/histology/V11U23-321_D1.tif")

In [ ]:
plt.imshow(ST)#[::-1,::-1,:])

In [ ]:
A = a["PANCK","BCATENIN","SMA"].copy()

In [ ]:
# flip the MxIF image horizontally (if needed)
A = A[::-1,::-1,:]

In [ ]:
# flip the MxIF image horizontally (if needed)
ST = ST[::-1,::-1,:]

## Set up napari viewer

In [ ]:
import napari

In [ ]:
viewer = registration_viewer(bg_img=A, fg_img=ST)

## Extract applied affine transformation

In [ ]:
affine, scale, shape = extract_affine(viewer=viewer, fg_img=ST)

In [ ]:
affine

In [ ]:
scale

In [ ]:
shape

In [ ]:
# save affine matrix and final shape to files so we don't have to re-do the manual registration in the future
np.savetxt(
    X=affine,
    fname="MxIF_out/{}_downsample8_{}_affine.csv".format(slide_id,visium_id),
    delimiter=",",
)
np.savetxt(
    X=shape,
    fname="MxIF_out/{}_downsample8_{}_shape.csv".format(slide_id,visium_id),
    delimiter=",",
)

### Pull image out of background and export

In [ ]:
image = transform.warp(A, transform.AffineTransform(affine), output_shape=abs(shape))

In [ ]:
plt.imshow(image)